In [1]:
# WeatherPy
# Julie Baker
# June 2021

In [ ]:
# Dependencies

In [ ]:
# citipy data

In [ ]:
# open weather api

In [ ]:
# select >= 500 unique (non-repeat)

## SCATTERPLOTS

### Temperature (F) vs. Latitude
* Plot
* 1-2 sentence explanation

### Humidity (%) vs. Latitude
* Plot
* 1-2 sentence explanation

### Cloudiness (%) vs. Latitude
* Plot
* 1-2 sentence explanation

### Wind Speed (mph) vs. Latitude
* Plot
* 1-2 sentence explanation

## REGRESSIONS
### Explain patterns after each plot

### Temperature (F) vs. Latitude: Northern Hemisphere

### Temperature (F) vs. Latitude: Southern Hemisphere

### Humidity (%) vs. Latitude: Northern Hemisphere

### Humidity (%) vs. Latitude: Southern Hemisphere

### Cloudiness (%) vs. Latitude: Northern Hemisphere

### Cloudiness (%) vs. Latitude: Southern Hemisphere

### Wind Speed (mph) vs. Latitude: Northern Hemisphere

### Wind Speed (mph) vs. Latitude: Southern Hemisphere